In [4]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))


In [5]:
from engines.cash_flow_engine import cash_flow_engine

import pandas as pd
financials = pd.read_csv("../data/cleaned/financial_statements.csv")
cash_flow_results = cash_flow_engine(financials)
cash_flow_results[:2]


[{'engine': 'cash_flow_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': np.int64(2020),
  'metrics': {'operating_profit': np.int64(813322),
   'coverage_proxy': np.float64(0.2776502541217982)},
  'flags': {'negative_operating_profit': np.False_,
   'weak_coverage': np.True_,
   'severity': 'watch'},
  'explanation': 'Cash generation shows signs of pressure and warrants monitoring.'},
 {'engine': 'cash_flow_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': np.int64(2021),
  'metrics': {'operating_profit': np.int64(920171),
   'coverage_proxy': np.float64(0.38882463675949536)},
  'flags': {'negative_operating_profit': np.False_,
   'weak_coverage': np.True_,
   'severity': 'watch'},
  'explanation': 'Cash generation shows signs of pressure and warrants monitoring.'}]